In [1]:
#| default_exp  redis

In [2]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
#| export
import redis.asyncio as aioredis
from pydantic import BaseModel
from typing import Optional,Union,List, Literal
from dataclasses import dataclass
import redis
import json
import pandas as pd


In [5]:
#| export

async def get_inner_redis():
    
    db = await aioredis.from_url(
            "redis://redis:6379/0",decode_responses=True
        )

    await db.ping()

    return db

In [6]:
#| export

async def get_stream_redis():
    
    db = await aioredis.from_url(
            "redis://host.docker.internal:6380/0",decode_responses=True
        )

    await db.ping()

    return db

In [7]:
# from host

async def get_stream_redis():
    
    db = await aioredis.from_url(
            "redis://localhost:6380/0",decode_responses=True
        )

    await db.ping()

    return db

In [8]:
#| export

async def get_connection_queues(name, redis_client, min_length=1, pattern='*'):
    """
    Finds queues that match a given pattern and have a minimum number of items,
    and returns a DataFrame with queue names and their lengths.
    """
    matching_queues = []
    queue_lengths = []
    cursor = '0'
    while cursor != 0:
        cursor, keys = await redis_client.scan(cursor, match=f'{name}:{pattern}', count=min_length)
        for key in keys:
            if await redis_client.type(key) == 'list' and await redis_client.llen(key) >= min_length:
                matching_queues.append(key)
                length = await redis_client.llen(key)
                queue_lengths.append(length)

    data = {
        'queue': matching_queues,
        'length': queue_lengths
    }
    return pd.DataFrame(data)


/tmp/ipykernel_66261/648225756.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [21]:
redis_inner_client = await get_inner_redis()
await get_connection_queues('Audio2DiarizeQueue',redis_inner_client)

,queue,length


In [9]:
redis_client = await get_redis()

In [10]:
await redis_client.ping()

True

In [27]:
#| export
class StreamItem(BaseModel):
    chunk: bytes
    timestamp: Optional[str] = None

    def dump(self):
        chunk = self.chunk.hex()
        self.timestamp =  str(datetime.utcnow())


        return {
                "chunk": chunk,
                "timestamp":self.timestamp
                }

    @classmethod
    def load(cls, item):
        item = json.loads(item)
        chunk = bytes.fromhex(item['chunk'])
        return cls(chunk=bytes.fromhex(item['chunk']), timestamp=item['timestamp'])


In [28]:
#| export
class ApiConnection:
    """
    connection to initial chrome extention feed
    """
    def __init__(self, conn_id, redis_client):
        self.id = conn_id
        self.redis_client = redis_client
        self.name = f"Connection:{self.id}"
        self.start_timestamp = pd.Timestamp.utcnow()
        self.queue = f"initialFeed_audio:{self.id}"


    async def set_open(self):
            self.open = True
            await self.set_update()

    async def set_closed(self):
        self.open = False
        await self.set_update()

    async def set_update(self):
        await self.redis_client.hset(self.name, 'open', int(self.open))

    async def enqueue(self, message):
        await self.redis_client.lpush(self.queue, json.dumps(message))

    async def check_open(self):
        self.open = bool(int(await self.redis_client.hget(self.name, 'open')))
        return self.open
    
    async def delete(self):
        await self.redis_client.delete(self.name)
        await self.redis_client.delete(self.queue)

    async def pop_multiple(self, max_len, min_len=1):
        lua_script = """
        local items = {}
        for i = 1, tonumber(ARGV[1]) do
            local item = redis.call('rpop', KEYS[1])
            if item == nil then
                break
            end
            table.insert(items, item)
        end
        return items
        """
        if await self.redis_client.llen(self.queue) >= min_len:
            items = await self.redis_client.eval(lua_script, 1, self.queue, max_len)
            items = [i for i in items if i]
            return [StreamItem.load(i) for i in items]
        else:
            print('items not available')
        
    async def get_queue_length(self):
        return await self.redis_client.llen(self.queue)
    
    async def get_all_queue(self):
        items =  await self.redis_client.lrange(self.queue, 0, -1)
        items = [i for i in items if i]
        return [StreamItem.load(i) for i in items]
    

    



In [31]:
#| export
@dataclass
class Data:
    chunk_name:str
    key:Literal['audio','transcription','diarisation']
    redis_client: redis.client.Redis
    data: Union[List, bytes]=None
    

    async def save(self):
            if isinstance(self.data, bytes): 
                data = self.data.hex()
            else: 
                data = json.dumps(self.data)
            await self.redis_client.hset(self.chunk_name, self.key, data)

    async def get(self):
        data = await self.redis_client.hget(self.chunk_name, self.key)
        if data:
            if self.key == 'audio': 
                self.data = bytes.fromhex(data)
            else: 
                self.data = json.loads(data)
        else: 
            self.data = None
        return bool(data)

    
    async def delete(self):
        return bool(await self.redis_client.delete(self.chunk_name))



class Audio(Data):
    def __init__(self, chunk_name: str, redis_client: redis.client.Redis, data: bytes = None):
        super().__init__(chunk_name=chunk_name, key='audio', redis_client=redis_client, data=data)

    async def fix(self):
        self.data = await asyncio.to_thread(fix_mp3_data, self.data)
        

class Transcript(Data):
    def __init__(self, chunk_name: str, redis_client: redis.client.Redis, data: List = None):
        super().__init__(chunk_name=chunk_name, key='transcription', redis_client=redis_client, data=data)

class Diarisation(Data):
    def __init__(self, chunk_name: str, redis_client: redis.client.Redis, data: List = None):
        super().__init__(chunk_name=chunk_name, key='diarisation', redis_client=redis_client, data=data)
